In [49]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
Show/hide the code by clicking <a href="javascript:code_toggle()">here</a>.''')

In [50]:
%matplotlib inline

##### don't show warnings
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

##### import packages
from brian2 import *
from brian2.units.constants import zero_celsius, gas_constant as R, faraday_constant as F
import numpy as np
import pandas as pd
pd.set_option('display.width', 115)
import matplotlib.pyplot as plt
import os
from IPython.display import Markdown as md

##### sets working directory
os.chdir("C:/Users/Richard/Documents/Studium/Master Elektrotechnik/Semester 4/Python/Models Brian2")

##### import models
import models.Rattay_2001 as rattay_01
import models.Frijns_1994 as frijns_94
import models.Frijns_2005 as frijns_05
import models.Smit_2009 as smit_09
import models.Smit_2010 as smit_10
import models.Imennov_2009 as imennov_09
import models.Negm_2014 as negm_14

##### makes code faster and prevents warning
prefs.codegen.target = "numpy"

# =============================================================================
# Load data
# =============================================================================
##### choose model
model = smit_10

##### load datasets
strength_duration_data = pd.read_csv(f'test_battery_results/{model.display_name}/Strength_duration_data {model.display_name}.csv')
threshold_table = pd.read_csv(f'test_battery_results/{model.display_name}/Threshold_table {model.display_name}.csv')
relative_spreads = pd.read_csv(f'test_battery_results/{model.display_name}/Relative_spreads {model.display_name}.csv')
conduction_velocity_table = pd.read_csv(f'test_battery_results/{model.display_name}/Conduction_velocity_table {model.display_name}.csv')
node_response_data_summary = pd.read_csv(f'test_battery_results/{model.display_name}/Node_response_data_summary {model.display_name}.csv')
refractory_table = pd.read_csv(f'test_battery_results/{model.display_name}/Refractory_table {model.display_name}.csv')

##### get plot links
strength_duration_curve_link = f"test_battery_results/{model.display_name}/Strength_duration_curve {model.display_name}.png"
relative_spread_plot_link = f"test_battery_results/{model.display_name}/Relative_spreads_plot {model.display_name}.png"
single_node_response_link = f"test_battery_results/{model.display_name}/Single_node_response {model.display_name}.png"
refractory_curve_link = f"test_battery_results/{model.display_name}/Refractory_curve {model.display_name}.png"
post_stimulus_time_histogram_link = f"test_battery_results/{model.display_name}/PSTH {model.display_name}.png"

# =============================================================================
# Add experimental results to tables
# =============================================================================
##### Strength duration datdisplay_namea
strength_duration_data = strength_duration_data.transpose()
strength_duration_data = strength_duration_data.rename(index = str, columns={0:"model"})
strength_duration_data["model"] = ["%.3g" %strength_duration_data["model"][i] for i in range(0,2)]
strength_duration_data["Van den Honert and Stypulkowski 1984"] = ["95.8", "247"]

##### Relative spread of thresholds
relative_spreads = relative_spreads.rename(index = str, columns={"relative spread":"model"})
relative_spreads["experiments"] = ["6.3%","5-10%","12%","11%"]
relative_spreads["reference"] = ["Miller et al. 1999","Dynes 1996","Javel et al. 1987","Javel et al. 1987"]
relative_spreads = relative_spreads.set_index(["phase duration","pulse form"])

##### Conduction velocity
conduction_velocity_table = conduction_velocity_table.transpose()
conduction_velocity_table = conduction_velocity_table.rename(index = str, columns={0:"model"})
if hasattr(model, "index_soma"):
    conduction_velocity_table["Hursh 1939"] = ["-","6","-","-"]
    if model.diameter_dendrite < 12*um:
        dendrite_ratio = 4.6
    else:
        dendrite_ratio = 5.66
    conduction_velocity_table["Boyd and Kalu 1979"] = ["-",f"{dendrite_ratio}","-","-"]
    conduction_velocity_table["Czèh et al 1976"] = ["-","-","v_ax = 0.9*v_den-6.9*m/s","-"]

else:
    conduction_velocity_table["Hursh 1939"] = ["-","6"]
    if model.diameter_fiber < 12*um:
        dendrite_ratio = 4.6
    else:
        dendrite_ratio = 5.66
    conduction_velocity_table["Boyd and Kalu 1979"] = ["-",f"{dendrite_ratio}"]

##### Latency and jitter
latency_jitter = node_response_data_summary[["phase duration (us)", "pulse form", "stimulus amplitude level", "latency (ms)",  "jitter (ms)"]]
latency_jitter = latency_jitter.rename(index = str, columns={"latency (ms)":"latency (us)", "jitter (ms)":"jitter (us)"})
latency_jitter = pd.melt(latency_jitter, id_vars=["phase duration (us)", "pulse form", "stimulus amplitude level"], value_vars=["latency (us)",  "jitter (us)"])
latency_jitter["value"] = latency_jitter["value"]*1000
latency_jitter["value"] = ["%.3g" %latency_jitter["value"][i] for i in range(0,latency_jitter.shape[0])]
latency_jitter["phase duration (us)"] = ["%g us" %latency_jitter["phase duration (us)"][i] for i in range(0,latency_jitter.shape[0])]
latency_jitter = latency_jitter.rename(index = str, columns={"phase duration (us)":"phase duration"})

latency_jitter_th = latency_jitter[latency_jitter["stimulus amplitude level"] == "threshold"]
latency_jitter_th = latency_jitter_th.rename(index = str, columns={"value":"model (threshold)"})

latency_jitter_2th = latency_jitter[latency_jitter["stimulus amplitude level"] == "2*threshold"]
latency_jitter_2th = latency_jitter_2th.rename(index = str, columns={"value":"model (2*threshold)"})

latency_jitter = latency_jitter.drop(columns = ["stimulus amplitude level", "value"]).drop_duplicates()
latency_jitter["model (threshold)"] = latency_jitter_th["model (threshold)"].tolist()
latency_jitter["model (2*threshold)"] = latency_jitter_2th["model (2*threshold)"].tolist()
latency_jitter = latency_jitter.sort_values(by=["pulse form", "phase duration"], ascending = [False, True])

latency_jitter["Miller et al. 1999"] = ["650", "100", "-", "-", "-", "-", "-", "-"]
latency_jitter["Van den Honert and Stypulkowski 1984 (threshold)"] = ["-", "-", "-", "-", "685", "352", "-", "-"]
latency_jitter["Van den Honert and Stypulkowski 1984 (2*threshold)"] = ["-", "-", "-", "-", "352", "8", "-", "-"]
latency_jitter["Hartmann and al. 1984"] = ["-", "-", "-", "-", "-", "-", "300-400", "-"]
latency_jitter["Cartee et al. 2000 (threshold)"] = ["-", "-", "440", "80", "-", "-", "-", "-"]

latency_jitter = latency_jitter.rename(index = str, columns={"variable":"property"})
latency_jitter = latency_jitter.set_index(["phase duration","pulse form", "property"])

##### AP shape
AP_shape = node_response_data_summary[["AP height (mV)", "rise time (ms)", "fall time (ms)", "AP duration (ms)"]].iloc[[5]].transpose()
AP_shape = AP_shape.rename(index = str, columns={5:"model"})

t_rise = round(-0.000625*conduction_velocity_table["model"].iloc[[0]] + 0.14, 3).tolist()[0]
t_fall = round(-0.002083*conduction_velocity_table["model"].iloc[[0]] + 0.3933, 3).tolist()[0]
AP_duration = t_rise + t_fall
AP_shape["Paintal 1965"] = ["-", t_rise, t_fall, AP_duration]

##### Refractory periods
absolute_refractory_periods = refractory_table.drop(columns = ["relative refractory period (ms)"])
absolute_refractory_periods = absolute_refractory_periods.rename(index = str, columns={"absolute refractory period (ms)":"ARP model (us)"})
absolute_refractory_periods["ARP model (us)"] = absolute_refractory_periods["ARP model (us)"]*1000
absolute_refractory_periods["ARP Experiments (us)"] = ["334","300","500-700","400-500","-"]
absolute_refractory_periods["reference"] = ["Miller et al. 2001","Stypulkowski and Van den Honert 1984","Dynes 1996","Brown and Abbas 1990", "-"]
absolute_refractory_periods = absolute_refractory_periods[absolute_refractory_periods["ARP Experiments (us)"] != "-"]
absolute_refractory_periods = absolute_refractory_periods.set_index(["phase duration (us)","pulse form"])

relative_refractory_periods = refractory_table.drop(columns = ["absolute refractory period (ms)"])
relative_refractory_periods = relative_refractory_periods.rename(index = str, columns={"relative refractory period (ms)":"RRP model (ms)"})
relative_refractory_periods["RRP Experiments (ms)"] = ["-","3-4; 4-5","5","-","5"]
relative_refractory_periods["reference"] = ["-","Stypulkowski and Van den Honert 1984; Cartee et al. 2000","Dynes 1996","-", "Hartmann et al. 1984"]
relative_refractory_periods = relative_refractory_periods[relative_refractory_periods["RRP Experiments (ms)"] != "-"]
relative_refractory_periods = relative_refractory_periods.set_index(["phase duration (us)","pulse form"])


In [51]:
md("# %s: Test battery results"%(model.display_name))

# Smit et al. 2010: Test battery results

## 1. Strength duration curve

In [52]:
md("![title](%s?arg)"%(strength_duration_curve_link))

![title](test_battery_results/Smit et al. 2010/Strength_duration_curve Smit et al. 2010.png?arg)

In [53]:
strength_duration_data

,model,Van den Honert and Stypulkowski 1984
rheobase (uA),0.152,95.8
chronaxie (us),165,247


## 2. Relative spread of thresholds

In [54]:
md("![title](%s?arg)"%(relative_spread_plot_link))

![title](test_battery_results/Smit et al. 2010/Relative_spreads_plot Smit et al. 2010.png?arg)

In [55]:
relative_spreads

,,model,experiments,reference
phase duration,pulse form,,,
40. us,mono,106.77%,6.3%,Miller et al. 1999
100. us,mono,135.23%,5-10%,Dynes 1996
200. us,bi,233.74%,12%,Javel et al. 1987
0.4 ms,bi,121.02%,11%,Javel et al. 1987


## 3. Conduction velocity

In [56]:
conduction_velocity_table

,model,Hursh 1939,Boyd and Kalu 1979,Czèh et al 1976
velocity dendrite (m/s),9.265,-,-,-
velocity/diameter dendrite,5.510,6,4.6,-
velocity axon (m/s),11.006,-,-,v_ax = 0.9*v_den-6.9*m/s
velocity/diameter axon,2.930,-,-,-


## 4. Single node response properties

In [57]:
md("![title](%s?arg)"%(single_node_response_link))

![title](test_battery_results/Smit et al. 2010/Single_node_response Smit et al. 2010.png?arg)

### 4.1 Action potential shape

In [58]:
AP_shape

,model,Paintal 1965
AP height (mV),134.000,-
rise time (ms),0.111,0.134
fall time (ms),1.060,0.374
AP duration (ms),1.170,0.508


### 4.2 Latency and jitter

In [59]:
latency_jitter

model (threshold) model (2*threshold) Miller et al. 1999  \
phase duration pulse form property                                                                
100 us         monophasic latency (us)               336                 180                650   
                          jitter (us)                100                16.6                100   
40 us          monophasic latency (us)               459                 138                  -   
                          jitter (us)                331                 8.6                  -   
50 us          monophasic latency (us)               473                 149                  -   
                          jitter (us)                362                10.1                  -   
200 us         biphasic   latency (us)               769                 355                  -   
                          jitter (us)                758                 494                  -   

                                       Van den Honert and Stypulkowski 1984 (threshold)  \
phase duration pulse form property                                                        
100 us         monophasic latency (us)                                                -   
                          jitter (us)                                                 -   
40 us          monophasic latency (us)                                                -   
                          jitter (us)                                                 -   
50 us          monophasic latency (us)                                              685   
                          jitter (us)                                               352   
200 us         biphasic   latency (us)                                                -   
                          jitter (us)                                                 -   

                                       Van den Honert and Stypulkowski 1984 (2*threshold) Hartmann and al. 1984  \
phase duration pulse form property                                                                                
100 us         monophasic latency (us)                                                  -                     -   
                          jitter (us)                                                   -                     -   
40 us          monophasic latency (us)                                                  -                     -   
                          jitter (us)                                                   -                     -   
50 us          monophasic latency (us)                                                352                     -   
                          jitter (us)                                                   8                     -   
200 us         biphasic   latency (us)                                                  -               300-400   
                          jitter (us)                                                   -                     -   

                                       Cartee et al. 2000 (threshold)  
phase duration pulse form property                                     
100 us         monophasic latency (us)                              -  
                          jitter (us)                               -  
40 us          monophasic latency (us)                            440  
                          jitter (us)                              80  
50 us          monophasic latency (us)                              -  
                          jitter (us)                               -  
200 us         biphasic   latency (us)                              -  
                          jitter (us)                               -

## 5. Recfractory properties

In [60]:
md("![title](%s?arg)"%(refractory_curve_link))

![title](test_battery_results/Smit et al. 2010/Refractory_curve Smit et al. 2010.png?arg)

### 5.1 Absolute refractory periods

In [61]:
absolute_refractory_periods

,,ARP model (us),ARP Experiments (us),reference
phase duration (us),pulse form,,,
40.0,monophasic,1891.0,334,Miller et al. 2001
50.0,monophasic,1886.0,300,Stypulkowski and Van den Honert 1984
100.0,monophasic,1862.0,500-700,Dynes 1996
50.0,biphasic,1860.0,400-500,Brown and Abbas 1990


### 5.2 Relative refractory periods

In [62]:
relative_refractory_periods

,,RRP model (ms),RRP Experiments (ms),reference
phase duration (us),pulse form,,,
50.0,monophasic,2.342,3-4; 4-5,Stypulkowski and Van den Honert 1984; Cartee e...
100.0,monophasic,2.308,5,Dynes 1996
200.0,biphasic,2.025,5,Hartmann et al. 1984


## 6. Post-stimulus time histogramm

In [63]:
md("![title](%s?arg)"%(post_stimulus_time_histogram_link))

![title](test_battery_results/Smit et al. 2010/PSTH Smit et al. 2010.png?arg)

In [64]:
%%HTML
<style type="text/css">
table.dataframe td, table.dataframe th {
    border-right: 1px  black solid !important;
    text-align: center !important;
    color: black !important;
}